In [1]:
import tensorflow as tf
from tf_agents.trajectories import time_step as ts
import tf_agents.trajectories as tr
import numpy as np
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.environments import tf_py_environment


2022-04-07 22:57:44.615163: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-07 22:57:44.615180: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
class RunEnv(py_environment.PyEnvironment):
    def __init__(self):
        self.obs = [ 0.0 for x in range(87*2) ]
        self._observation_max = []
        self._observation_min = []
        for i in range(87):
            self._observation_max.append(1) # max value for object label in view
            self._observation_min.append(0)
        for i in range(87):
            self._observation_max.append(8.0)
            self._observation_min.append(0)
        self._observation_max = np.array(self._observation_max)
        self._observation_min = np.array(self._observation_min)
        self._observation_spec = array_spec.BoundedArraySpec(
        shape=((87)*2, ), dtype=np.float32, minimum=self._observation_min, maximum=self._observation_max, name='observation')
        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int32,minimum=0,maximum=2,name='action')
        self._done=False
    def action_spec(self):
        return self._action_spec
    def observation_spec(self):
        return self._observation_spec
    def _step(self, action):
        if self._done:
            return ts.termination(np.array(self.obs, dtype=np.float32), reward=0.0)
        else:            
            return ts.transition(np.array(self.obs, dtype=np.float32), reward=0.0, discount=0.0)
    def _reset(self):
        return ts.restart(np.array(self.obs, dtype=np.float32))
        


# Load Trained Policy

In [3]:
saved_policy = tf.compat.v2.saved_model.load('example_policy')
policy_state = saved_policy.get_initial_state(batch_size=1)

2022-04-07 22:57:46.012829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-07 22:57:46.012867: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-07 22:57:46.012878: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2022-04-07 22:57:46.012999: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Set initial Observation

In [9]:
obs = [ 0.0 for x in range(87*2) ]
runenv = RunEnv()
env = tf_py_environment.TFPyEnvironment(runenv)
runenv.obs = obs
time_step = env.reset()
policy_step = saved_policy.action(time_step, policy_state)
policy_state = policy_step.state
print(policy_step.action)
print(int(policy_step.action))

tf.Tensor([2], shape=(1,), dtype=int32)
2


In [10]:
runenv.obs = [ 1.0 for x in range(87*2) ]
time_step = env.reset()
policy_step = saved_policy.action(time_step, policy_state)
policy_state = policy_step.state
print(policy_step.action)
print(int(policy_step.action))

tf.Tensor([1], shape=(1,), dtype=int32)
1
